In [1]:
import torch
from torch import nn

import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

import IPython.display as idp

import matplotlib.pyplot as plt
import numpy as np


In [2]:
visualize=False

### 1) Input and preparation

In [3]:
# load audio

sample_rate = 44100 # 

def show_idp_audio(waveform):
    n = 14
    return idp.display(idp.Audio(waveform[(3 * n) * sample_rate:(3 * (n + 1)) * sample_rate], rate=sample_rate))

def load_audio(path, visualize=False):
    waveform, sr = torchaudio.load(path)
    # Convert everthing to mono channel for simplicity
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0)
        # waveform is now a vector 
    # Resample everything to 44.1khz for simplicity
    resampler = T.Resample(sr, sample_rate, dtype=waveform.dtype)
    waveform = resampler(waveform)
    
    if visualize:
        # samplerate = 1/t
        # display the first 3 seconds
        show_idp_audio(waveform)
    
    return waveform

if visualize:
    sample_waveform = load_audio("mixture.wav", visualize=True)

    sample_waveform.size()

In [4]:
def rms_normalize(waveform, target_rms):
    current_rms = torch.sqrt(torch.mean(waveform**2))
    gain_factor = target_rms / (current_rms + 1e-10)
    normalized_waveform = waveform * gain_factor
    return normalized_waveform, gain_factor

def rms_denormalize(normalized_waveform, gain_factor):
    inverse_gain = 1 / gain_factor
    reversed_waveform = normalized_waveform * inverse_gain
    return reversed_waveform

def peak_normalize(waveform, target_peak):
    peak_value = torch.max(torch.abs(waveform))
    peak_gain_factor = target_peak / (peak_value + 1e-10)
    normalized_waveform = waveform * peak_gain_factor
    return normalized_waveform, peak_gain_factor

def peak_denormalize(normalized_waveform, peak_gain_factor):
    inverse_peak_gain = 1 / peak_gain_factor
    reversed_waveform = normalized_waveform * inverse_peak_gain
    return reversed_waveform

def inspect_waveform(waveform):
    transform = T.Loudness(sample_rate)
    return f"LKFS:{transform(waveform.unsqueeze(0))} max: {waveform.max()} min: {waveform.min()} avg: {waveform.mean()}"

def normalize_waveform(waveform, visualize=False):
    """ rms -> peak """
    # target rms can be anything. the important part here
    # is to be constant for all kind of songs

    if visualize:
        print("original: " + inspect_waveform(waveform))
        show_idp_audio(waveform)

    normalized_waveform, gain_factor = rms_normalize(waveform, target_rms=0.1)
    
    if visualize:
        print("rms_normalize: " + inspect_waveform(normalized_waveform))
        show_idp_audio(normalized_waveform)

    # setting target peak to 1.0 forces the values between -1.0 < y < 1.0
    normalized_waveform, peak_gain_factor = peak_normalize(normalized_waveform, target_peak=0.1)
    
    if visualize:
        print("peak_normalize: " + inspect_waveform(normalized_waveform))
        show_idp_audio(normalized_waveform)
    
    return normalized_waveform, gain_factor, peak_gain_factor

def de_normalize_waveform(waveform, gain_factor, peak_gain_factor, visualize=False):
    if visualize:
        print("de_normalize_waveform: " + inspect_waveform(waveform))
        show_idp_audio(waveform)
    
    waveform = peak_denormalize(waveform, peak_gain_factor)
    
    if visualize:
        print("peak_denormalize: " + inspect_waveform(waveform))
        show_idp_audio(waveform)
    waveform = rms_denormalize(waveform, gain_factor)
    
    if visualize:
        print("rms_denormalize: " + inspect_waveform(waveform))
        show_idp_audio(waveform)
    
    return waveform

if visualize:
    normal_waveform, gain_factor, peak_gain_factor = normalize_waveform(sample_waveform, visualize=True)
    _ = de_normalize_waveform(normal_waveform, gain_factor, peak_gain_factor, visualize=True)

In [5]:
chunk_size_in_seconds = 1
chunk_size = chunk_size_in_seconds * sample_rate

def split(waveform, visualize=False):
    # we have a vector by length n and we want to split it to even chunks by length of
    # chunk_size
    padding_length = (chunk_size - waveform.shape[0] % chunk_size) % chunk_size
    waveform = nn.functional.pad(waveform, (0, padding_length), 'constant', 0)
    # -1 means automatically infer based on other dims
    chunked_waveform = waveform.view(-1, chunk_size)
    
    if visualize:
        fig = plt.figure(constrained_layout=True, figsize=(16, 4))
        subfigs = fig.subfigures(2, 1).flat
        
        # first 3 chunk_size of waveform
        w = waveform[:3 * chunk_size].detach().numpy()
        ylim = [w.max() * 1.1, w.min() * 1.1]
        def time_axis(start, duration):
            return torch.arange(start * sample_rate, (duration + start) * sample_rate) / sample_rate
        axes = subfigs[0].subplots(1, 1)
        axes.plot(time_axis(0, 3), w, linewidth=0.3)
        axes.set_xlabel("time [s] for first 3 seconds")
        axes.set_ylim(ylim)
        
        # first 4 chunks + last chunk
        axes = subfigs[1].subplots(1, 5)
        for i, chunk in enumerate([0, 1, 3, 4, chunked_waveform.shape[0] - 1]): 
            axes[i].plot(time_axis(0, chunk_size_in_seconds), chunked_waveform[chunk], linewidth=0.3)
            axes[i].set_title(f"chunk {chunk}")
            axes[i].set_ylim(ylim)
        
    return chunked_waveform, padding_length

def merge(chunks, padding_length):
    merged_waveform = torch.cat([torch.flatten(x) for x in chunks])
    return merged_waveform[:-padding_length]

if visualize:
    sample_waveform_chunks, padding_length = split(normal_waveform, visualize=True)

    assert sample_waveform_chunks.shape[1] == chunk_size

    sample_merged = merge(sample_waveform_chunks, padding_length)

    assert sample_merged.shape == normal_waveform.shape
    assert torch.all(sample_merged == normal_waveform)

In [6]:
n_fft = 2048
win_length = n_fft
hop_length = win_length // 4


def visualize_spectogram(chunk, chunk_stft, title='Spectogram'):
    import librosa
    fig, axis = plt.subplots(2, 1, figsize=(16, 5))
    noverlap = win_length - hop_length
    axis[0].imshow(librosa.power_to_db(chunk_stft.abs().detach().numpy() ** 2), origin="lower", aspect="auto", interpolation="nearest")
    axis[0].set_yscale("symlog")
    axis[0].set_title(title)
    if chunk is not None:
        axis[1].plot(chunk, linewidth=0.5)
        axis[1].grid(True)
        axis[1].set_xlim([0, len(chunk)])

def to_spectogram():
    transform_spectogram = T.Spectrogram(n_fft=n_fft, win_length=win_length, hop_length=hop_length,\
                                         window_fn=torch.hamming_window, power=None)
    
    def inner(chunk, visualize=False, title=''):
        chunk_stft = transform_spectogram(chunk)
        if visualize:
            visualize_spectogram(chunk, chunk_stft, title)

        return chunk_stft
    return inner

if visualize:
    print(sample_waveform_chunks[1].shape)
    chunk_stft = to_spectogram()(sample_waveform_chunks[1], visualize=True)
    chunk_stft.shape

In [7]:
class Vis:
    def visualize(self, input):
        from torchview import draw_graph
        y = self(input)
        x = draw_graph(self, input_data=input, device='meta', roll=True)
        print(f"--{input.shape}-->f(x)--{y.shape}-->")
        file = x.visual_graph.render(self._get_name())
        display(idp.FileLink("./" + file))

In [8]:
# Numbers are exctracted from the paper
splits_v7 = [
   # below 1kh, bandwidth 100hz
   (1000, 100),
   # above 1kh and below 4khz, bandwidth 250hz
   (4000, 250),
   (8000, 500),
   (16000, 1000),
   (20000, 2000),
]

temporal_dim = int(np.ceil(chunk_size / T.Spectrogram(n_fft=n_fft, win_length=win_length, hop_length=hop_length).hop_length))
feature_dim = 128 // 4

# Module 1
class BandSplit(nn.Module, Vis):
    
    def __init__(self, splits=splits_v7, fully_connected_out=feature_dim):
        super(BandSplit, self).__init__()
        
        
        #### Make splits
        # convert fft to freq
        freqs = sample_rate * torch.fft.fftfreq(n_fft)[:n_fft // 2 + 1]
        freqs[-1] = sample_rate // 2
        indices = []
        start_freq, start_index = 0, 0
        for end_freq, step in splits:
            bands = torch.arange(start_freq + step, end_freq + step, step)
            start_freq = end_freq
            for band in bands:
                end_index = freqs[freqs < band].shape[0]
                indices.append((start_index, end_index))
                start_index = end_index
        indices.append((start_index, freqs.shape[0]))
        self.band_indices = indices
        self.fully_connected_out = fully_connected_out
        
        self.layer_norms = nn.ModuleList([
            # * 2 is for added dim of view_as_real
            nn.LayerNorm([(band_end - band_start) * 2, temporal_dim])
            for band_start, band_end in self.band_indices
        ])
        
        self.layer_fcs =  nn.ModuleList([
            # * 2 is for added dim of view_as_real
            nn.Linear((band_end - band_start) * 2, fully_connected_out)
            for band_start, band_end in self.band_indices
        ])

    def forward(self, chunk_ftt):
        batch_size = chunk_ftt.size(0)
        stack = []
        # TODO: can i vectorize this loop?
        for i, (band_start, band_end) in enumerate(self.band_indices):
            band = chunk_ftt[:, band_start:band_end, :]
            # band is shape of (B, F, T)
            band = torch.view_as_real(band) # (B, F, T, 2)
            # convert to (B, 2, F, T) to be able to feed it to the norm
            band = band.permute(0, 3, 1, 2)
            
            # norm is (..., F, T) and fc is (Fxfully_connected_out)
            # we should make norm (..., T, F) in order to feed it to the fc
            band = band.reshape(batch_size, -1, band.size(-1)) # -1 = T
            norm = self.layer_norms[i](band)
            
            norm = norm.transpose(-1, -2).contiguous()
            fc_y = self.layer_fcs[i](norm)
            
            stack.append(fc_y)
        return torch.stack(stack, dim=1)

if visualize:
    bandsplit_layer = BandSplit()
    bandsplit_y = bandsplit_layer(chunk_stft.unsqueeze(0))
    bandsplit_layer.visualize(chunk_stft.unsqueeze(0))

In [9]:
# Module 2

class RNN(nn.Module):
    def __init__(self, input_dim_size):
        super(RNN, self).__init__()
        self.input_dim_size = input_dim_size
        # paper specified group norm
        self.norm = nn.ModuleList([nn.GroupNorm(self.input_dim_size, self.input_dim_size) for _ in range(2)])
        self.blstm = nn.ModuleList([nn.LSTM(self.input_dim_size, self.input_dim_size, bidirectional=True, batch_first=True) for _ in range(2)])
        self.fc = nn.ModuleList([nn.Linear(self.input_dim_size * 2, self.input_dim_size) for _ in range(2)])
    
    def forward(self, x):
        # input is b, bands(K), temporal_dim(t), input_dim_size
        
        
        # First loops converts the shape to [B, T, K, N]
        # and the second loop converts it back to [B, K, T, N]
        for i in range(2):
            B, K, T, N = x.shape
            out = x.view(B * K, T, N)
            out = self.norm[i](out.transpose(-1, -2)).transpose(-1, -2)
            out = self.blstm[i](out)[0]
            out = self.fc[i](out)
            x = out.view(B, K, T, N) + x
            x = x.permute(0, 2, 1, 3).contiguous()
        
        return x

num_blstm_layers=24 // 2

class BandSequence(nn.Module, Vis):
    
    def __init__(self, input_dim_size, num_layers=num_blstm_layers):
        super(BandSequence, self).__init__()
        self.rnns = nn.Sequential(*[RNN(input_dim_size=input_dim_size) for _ in range(num_layers)])
        
    def forward(self, x):
        # (bands, temporal_dim, fc_out)
        return self.rnns(x)

if visualize:
    bandsequence_layer = BandSequence(input_dim_size=bandsplit_layer.fully_connected_out)
    bandsequence_y = bandsequence_layer(bandsplit_y)
    bandsequence_layer.visualize(bandsplit_y)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

class MaskEstimation(nn.Module, Vis):
    def __init__(self, band_indices, fully_connected_out):
        super(MaskEstimation, self).__init__()
        
        max_indice_diff = max([e - s for s, e in band_indices])
        num_hiddens = lambda e, s: 3 * (max_indice_diff - (e - s) + 1)
        self.layers = nn.ModuleList([
            nn.Sequential(
                nn.LayerNorm([temporal_dim, fully_connected_out]),
                nn.Linear(fully_connected_out, num_hiddens(e, s)),
                nn.Tanh(),
                # double the output dim to use in GLU
                # the extra *2 is for returning as complex
                nn.Linear(num_hiddens(e, s), (e - s) * 2 * 2),
                nn.GLU()
            )
            for s, e in band_indices
        ])
        
    def forward(self, x):
        # (b, k, temporal_dim, fc_out)
        parts = []
        for i in range(x.shape[1]):
            y = self.layers[i](x[:, i]).contiguous()
            B, T, F = y.shape
            y = y.permute(0, 2, 1).contiguous() # B F T
            # basically halve the freq dim and use it for phasee
            y = y.view(B, 2, F // 2, T) # (B, 2, F, T)
            y = y.permute(0, 2, 3, 1) # (B, F, T, 2)
            y = torch.view_as_complex(y.contiguous())
            
            parts.append(y)
        
        # (b, f, t)
        return torch.cat(parts, dim=-2)

    
if visualize:   
    mask_layer = MaskEstimation(band_indices=bandsplit_layer.band_indices, fully_connected_out=bandsplit_layer.fully_connected_out)
    mask_y = mask_layer(bandsequence_y)
    mask_layer.visualize(bandsequence_y)

In [11]:
class BSRNN(nn.Module, Vis):
    
    def __init__(self):
        super(BSRNN, self).__init__()
        
        self.split = BandSplit()
        self.sequence = BandSequence(input_dim_size=self.split.fully_connected_out)
        self.mask = MaskEstimation(band_indices=self.split.band_indices, fully_connected_out=self.split.fully_connected_out)

    def forward(self, chunk_fft):
        
        mean = chunk_fft.mean(dim=(1, 2), keepdim=True)
        std = chunk_fft.std(dim=(1, 2), keepdim=True)
        chunk_fft = (chunk_fft - mean) / (std + 1e-5)
        
        y = self.split(chunk_fft)
        y = self.sequence(y)
        mask = self.mask(y)
        
        mask = mask * std + mean

        return mask

if visualize:
    bsrnn = BSRNN().cuda()
    bsrnn_y = bsrnn(chunk_stft.unsqueeze(0).cuda())
    bsrnn.visualize(chunk_stft.unsqueeze(0).cuda())

In [12]:

def from_spectogram():
    # We are using GriffinLim to ensure the output size
    transform_inv_spectogram = T.InverseSpectrogram(n_fft=n_fft, win_length=win_length, hop_length=hop_length,\
                                                    window_fn=torch.hamming_window)

    def inner(chunk_stft, visualize=False):
        chunk = transform_inv_spectogram(chunk_stft)
    
        if visualize:
            visualize_spectogram(chunk.detach().numpy(), chunk_stft)

        return chunk
    return inner

if visualize:
    print(bsrnn_y.shape)
    masked_complex = chunk_stft * bsrnn_y[0].cpu()

    chunk_y = from_spectogram()(masked_complex, visualize=True)
    chunk_y.shape

In [13]:
if visualize:
    chunk_x = de_normalize_waveform(sample_waveform_chunks[1], gain_factor, peak_gain_factor)
    chunk_y = de_normalize_waveform(chunk_y, gain_factor, peak_gain_factor)

    print("In:")
    show_idp_audio(chunk_x)
    print("Out:")
    show_idp_audio(chunk_y.detach().numpy())
    print(chunk_y.shape)
    chunk_size

In [14]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [15]:
# Put Everything Together! audio in, audio out!


class MSSBandSplitRNN(nn.Module, Vis):
    def __init__(self):
        super(MSSBandSplitRNN, self).__init__()
        self.to_spectogram = to_spectogram()
        self.from_spectogram = from_spectogram()
        self.bsrnn = BSRNN()
        
    def forward(self, waveform):
        """ Waveform in -> Waveform out :) """
        
        # 1) normalize
        # 2) split
        # 3) feed to bsrnn
        # 4) convert spectogram to audio
        # 5) merge all splits
        # 6) de-normalize
        
        normal_waveform, gain_factor, peak_gain_factor = normalize_waveform(waveform)
        splits, padding_length = split(normal_waveform)
        masked_splits = [() for _ in range(len(splits))]
        for i, x_split in enumerate(splits):
            split_stft = self.to_spectogram(x_split)
            mask = self.bsrnn(split_stft.unsqueeze(0))[0]
            
            masked_complex = mask
            
            wave = self.from_spectogram(masked_complex)
            masked_splits[i] = wave
        
        masked_waveform = merge(masked_splits, padding_length)
        y = de_normalize_waveform(masked_waveform, gain_factor, peak_gain_factor)
        return y

if visualize:
    torch.set_default_device('cuda')
    with torch.no_grad():
        model = MSSBandSplitRNN()
        y = model(sample_waveform.cuda())


    print("In:")
    show_idp_audio(sample_waveform)
    print("Out:")
    show_idp_audio(y.cpu().detach().numpy())

# 2) Train

In [16]:
class CustomLoss(nn.Module):
    
    def __init__(self):
        super(CustomLoss, self).__init__()
        self.mae_stft_real = nn.L1Loss()
        self.mae_stft_imag = nn.L1Loss()
        self.mae_inv_stft  = nn.L1Loss()
    
    def forward(self, pred_stft, target_stft,  pred_inv_stft, target_inv_stft):
        loss_r = self.mae_stft_real(pred_stft.real, target_stft.real)
        loss_i = self.mae_stft_imag(pred_stft.imag, target_stft.imag)
        loss_t = self.mae_inv_stft(pred_inv_stft, target_inv_stft)
        loss = loss_r + loss_i + loss_t
        return loss

def compute_usdr(pred, target, delta = 1e-7):
    if pred.shape[0] < target.shape[0]:
        padding = target.shape[0] - pred.shape[0]
        pred = torch.nn.functional.pad(pred, (0, padding), "constant", 0)
    num = torch.sum(torch.square(target))
    den = torch.sum(torch.square(target - pred))
    num += delta
    den += delta
    usdr = 10 * torch.log10(num / den)
    return usdr.mean()    

if visualize:
    torch.set_default_device('cpu')
    target_wf = load_audio("drums.wav")
    normal_target_wf, gain_factor, peak_gain_factor = normalize_waveform(target_wf)
    target_wf_chunks, padding_length = split(normal_target_wf)
    stft = to_spectogram()
    inv_stft = from_spectogram()
    print("target:")
    show_idp_audio(target_wf_chunks[1])
    target_stft = stft(target_wf_chunks[1], visualize=True, title="Target")
    print("input:")
    show_idp_audio(sample_waveform_chunks[1])
    sample_stft = stft(sample_waveform_chunks[1], visualize=True, title="Input")

    # x * mask = target_stft / x
    # f(x) = stft - 

    mask_stft = target_stft / sample_stft
    visualize_spectogram(inv_stft(mask_stft), mask_stft, title="Mask stft")
    print("Mask:")
    show_idp_audio(inv_stft(mask_stft))
    target_y = sample_stft * mask_stft
    print("Target_y: ")
    show_idp_audio(inv_stft(target_y))
    visualize_spectogram(inv_stft(target_y), target_y, title="Target_y stft calc")


    loss_fn = CustomLoss()
    loss = loss_fn.forward(target_y, target_stft, inv_stft(target_y), inv_stft(target_stft))
    print(f"loss: {loss}")

In [17]:
torch.cuda.empty_cache()
gc.collect()

0

In [18]:
# torch.set_default_device('cuda')
model = torch.load("./bsrnn-102-0.031533900648355484.pt", map_location="cuda")
# b, (mix_stft, mask_stft) = next(enumerate(data_loader))
# model.eval()
# traced = torch.jit.trace(model, mix_stft.to('cuda'))



# model = BSRNN()
# torch.save(model, "a.pt")
# print(model)
# torch._dynamo.reset()
#model = torch.compile(model, mode="reduce-overhead")

In [19]:
# torch.onnx.export(model,               # model being run
#                   mix_stft.to('cuda'),                         # model input (or a tuple for multiple inputs)
#                   "my_bsrnn.onnx",   # where to save the model (can be a file or file-like object)
#                   export_params=True,        # store the trained parameter weights inside the model file
#                   opset_version=10,          # the ONNX version to export the model to
#                   do_constant_folding=True,  # whether to execute constant folding for optimization
#                   input_names = ['input'],   # the model's input names
#                   output_names = ['output'], # the model's output names
#                   dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
#                                 'output' : {0 : 'batch_size'}})
# torch.jit.save(traced, "traced.pt")

In [20]:
import os
import time
from torch.profiler import profile, record_function, ProfilerActivity
import random
from torch.utils.tensorboard import SummaryWriter
import datetime
import math
import json
torch.cuda.empty_cache()
gc.collect()

batch_size=95

torch.set_default_device('cpu')

class Dataset(torch.utils.data.IterableDataset):
    def __init__(self, path="/home/sahand/BandSplit-RNN/musdb18hq/train"):
        super(Dataset, self).__init__()
        self.files = [entry for entry in os.scandir(path) if entry.is_dir()]
        random.shuffle(self.files)
    
    def iterator(self):
        torch.set_default_device('cpu')
        to_stft = to_spectogram()
        for i in range(self.start_index, self.end_index):
            d = self.files[i]
            mixture = f"{d.path}/mixture.wav"
            target = f"{d.path}/drums.wav"

            normal_mix, _, _ = normalize_waveform(load_audio(mixture))
            normal_target, _, _ = normalize_waveform(load_audio(target))

            normal_mix, _ = split(normal_mix)
            normal_target, _ = split(normal_target)

            for mix, target in zip(normal_mix, normal_target):
                mix_stft = to_stft(mix)
                target_stft = to_stft(target)
                # Accumulate STFTs in the batch
                yield (mix_stft, target_stft)

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is not None:
            total_files = len(self.files)
            per_worker = int(math.ceil(total_files / float(worker_info.num_workers)))
            self.start_index = worker_info.id * per_worker
            self.end_index = min(self.start_index + per_worker, total_files)
        else:
            self.start_index = 0
            self.end_index = len(self.files)

        return iter(self.iterator())

print("Setting up data loader")
data_loader = torch.utils.data.DataLoader(Dataset(), num_workers=3, 
                                          batch_size=batch_size, drop_last=True, 
                                          # pin_memory=True, pin_memory_device='cuda',
                                          prefetch_factor=4,
                                          persistent_workers=True)


torch.set_default_device('cuda')
print(f"model params: {len(list(model.parameters()))}")
lr = 1e-5
clip_grad_norm = 5
optimizer = torch.optim.Adam(model.parameters() ,lr=lr)
loss_fn = CustomLoss()

inv_stft_gpu = from_spectogram()
#name = "logs/reversed-mask/2023-12-18 05:56:45.991714"

n_epochs = 500
start_epoch = 103
name = f'logs/reversed-mask/{datetime.datetime.now()}'
writer = SummaryWriter(name)
print(f"logdir = {name}")
hparams = {
    "lr": lr,
    "capacity": len(list(model.parameters())),
    "clip_grad_norm": clip_grad_norm,
    # "eps": eps,
    "batch_size": batch_size,
    "sample_rate": sample_rate,
    "chunk_size_in_seconds": chunk_size_in_seconds,
    "chunk_size": chunk_size,
    "spectogram/n_fft": n_fft,
    "spectogram/win_length": win_length,
    "spectogram/hop_length": hop_length,
    "model/feature_dim": num_blstm_layers,
}
writer.add_text("Hyper parameters", f"{json.dumps(hparams)}")


max_b = 239
for e in range(n_epochs - start_epoch):
    epoch = e + start_epoch
    total_loss = 0
    total_usdr = 0
    total_n = 0
    loss = None
    start = time.time()
    # with torch.profiler.profile(
    #     schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
    #     on_trace_ready=torch.profiler.tensorboard_trace_handler('./logs/prof'),
    #     record_shapes=True,
    #     profile_memory=True,
    #     with_stack=True
    # ) as prof:
    
    for b, (mix_stft, mask_stft) in enumerate(data_loader):
        max_b = max(b, max_b)
        x = b + max_b * epoch
        # prof.step()
        #print(mix_stft.shape)
        mix_stft = mix_stft.to('cuda')
        mask_stft = mask_stft.to('cuda')
        b_total = max_b
        #with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, with_stack=True) as prof:
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            y_mask = model(mix_stft)

        #print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=100))
        # print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))
        inv_mask_stft = inv_stft_gpu(mask_stft)
        inv_y_mask = inv_stft_gpu(y_mask)
        loss = loss_fn(y_mask, mask_stft, inv_y_mask, inv_mask_stft)
        total_loss += loss
        total_n += 1
        usdr = compute_usdr(inv_y_mask, inv_mask_stft)
        total_usdr += usdr
        mv_avg = total_loss/total_n
        print(f"Epoch {epoch}/{n_epochs} {b}/{b_total}: loss={loss} avg loss={mv_avg} usdr={usdr}", end='\r\033[K\r')

        # backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
        # update weights
        optimizer.step()
        optimizer.zero_grad()

        writer.add_scalar('Loss/train', loss, x)
        writer.add_scalar('usdr/train', usdr, x)
        writer.add_scalar('mv avg loss/train', mv_avg, x)
    # break
    avg_loss = total_loss/total_n
    avg_usdr = total_usdr/total_n
    writer.add_scalar('Avg Loss/train', avg_loss, epoch)
    writer.add_scalar('Avg usdr/train', avg_usdr, epoch)
    print(f"epoch #{epoch} finished in {(time.time() - start) / 60}m. loss={avg_loss} usdr={avg_usdr}\n")
    if epoch % 2 == 0:
        torch.save(model, f"bsrnn-{epoch}-{loss}.pt")
        # try:
        #     m = MSSBandSplitRNN()
        #     m.bsrnn = m
        #     mix_valid = load_audio("mixture.wav")
        #     y = m(mix_valid.cuda())
        #     writer.add_audio(f'Audio/example for {epoch}', y, epoch, sample_rate=sample_rate)
        # except BaseException as error:
        #     print(f"error while writing audio example -> {error}")
writer.close()


2023-12-21 11:51:20.802442: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 11:51:20.802487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 11:51:20.803120: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 11:51:20.807526: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 11:51:21.326196: W tensorflow/compiler/tf2

Setting up data loader
model params: 698


logdir = logs/reversed-mask/2023-12-21 11:51:22.940533


/home/sahand/mambaforge/envs/bsrnn/lib/python3.10/site-packages/torch/utils/_device.py:77: UserWarning: ComplexHalf support is experimental and many operators don't support it yet. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392067780/work/aten/src/ATen/EmptyTensor.cpp:31.)
  return func(*args, **kwargs)


epoch #103 finished in 12.799456389745076m. loss=0.06061919406056404 usdr=0.416151136159896855614013672



KeyboardInterrupt: 

In [20]:
print(x)

628


In [ ]:
torch.set_default_device('cuda')
m = MSSBandSplitRNN()
model.eval()
m.bsrnn = model
mixture_wav = load_audio("mixture.wav")
target_wav = load_audio("drums.wav")
with torch.no_grad():
    y = m(mixture_wav.cuda())
show_idp_audio(mixture_wav)
show_idp_audio(y.cpu().detach().numpy())
show_idp_audio(target_wav)


In [ ]:


torch.set_default_device('cuda')
import glob
models = list(glob.glob("*.pt"))
models = sorted(models, key=lambda x: int(x.split("-")[1]))
mixture_wav = load_audio("mixture.wav")
target_wav = load_audio("drums.wav")
print("mixture:")
show_idp_audio(mixture_wav)
print("target:")
show_idp_audio(target_wav)

dw = None
for i, model_path in enumerate(models):
    if i % 5 != 0:
        continue
    model_bsrnn = torch.load(model_path, map_location="cuda")
    m = MSSBandSplitRNN()
    m.bsrnn = model_bsrnn
    w = torch.cat([t.view(-1) for t in list(m.parameters())])
    if dw is None:
        dw = nn.L1Loss()(torch.zeros_like(w), w)
    else:
        dw = nn.L1Loss()(dw, w)
    with torch.no_grad():
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            y = m(mixture_wav.cuda())
        usdr = compute_usdr(y, target_wav.cuda())
        print(f"{model_path} △w={dw} usdr={usdr}: ")
        
        show_idp_audio(y.cpu().detach().numpy())
